### Create AML SDK docker file and build and test image

To run this notebook, start it via a plain Jupyter notebook session on your Ubuntu_Azure_VM:

* login (ssh) into the VM:
```
loginname1@Ubuntu_Azure_VM:/datadrive01/prj/regularR_Realtime$ jupyter notebook --notebook-dir=$(pwd) --ip='*' --port=9000 --no-browser --allow-root
```
  
   
* then go to:    
http://Ubuntu_Azure_VM.eastus2.cloudapp.azure.com:9000/ 
  
  
https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes  
2019-03-11
Azure Machine Learning SDK for Python v1.0.18  
   
Note: As of 02/13/2019, latest versions are:  
Python 3.7.2 (https://www.python.org/downloads/, https://anaconda.org/anaconda/python)  
miniconda3 4.5.12 (https://hub.docker.com/r/continuumio/miniconda3/tags)  
AML SDK  v.1.0.15 (https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py)  
Ubuntu 19.04 (https://hub.docker.com/_/ubuntu?tab=tags)  

In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
%load_ext dotenv
# import dotenv
# print (dir(dotenv))

In [3]:
%%writefile .env

# Your docker login and image repository name
docker_login = 'georgedockeraccount'
image_tag = "aml-sdk_docker_image"
image_version = 'sdk.v1.0.55' # also set this in conda file below

Overwriting .env


In [4]:
%dotenv
docker_file_location = 'docker_build'

import os
docker_file_name = 'dockerfile'+ '_' + os.getenv('image_tag') + '_' + os.getenv('image_version')
conda_dependency_file = 'aml_sdk_conda_dep_file.yml'

In [5]:
local_dir = !pwd
docker_file_dir = os.path.join(*([local_dir.s, docker_file_location]))
docker_file_dir
 
docker_file_path = os.path.join(*([docker_file_dir]+[docker_file_name]))
docker_file_path

conda_dependency_file_path = os.path.join(*([docker_file_dir]+[conda_dependency_file]))
conda_dependency_file_path

#clean previous history (i.e. content of amlsdk/docker)
!mkdir -p {docker_file_dir}
!chmod -R ugo=rwx $docker_file_dir
!rm -rf $docker_file_dir/*
!ls -l $docker_file_dir

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.55'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/aml_sdk_conda_dep_file.yml'

total 0


In [6]:
%%writefile $conda_dependency_file_path
name: aml_sdk_conda_env02
    
channels:
  - anaconda
dependencies:
  - python=3.6 # 3.6 req by tf, which seems to be req by automl, not 3.7.2 
  - numpy
  - cython
  - notebook 
  - nb_conda
  - scikit-learn
  - pip
  - pip:
    - python-dotenv
    - papermill[azure]
    - azureml-sdk[notebooks,automl,explain]==1.0.55 

Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/aml_sdk_conda_dep_file.yml


In [7]:
%%writefile $docker_file_path 

# FROM continuumio/miniconda3:4.5.12 as AzureMLSDKOnly 
FROM continuumio/miniconda3:4.6.14 as AzureMLSDKOnly    
MAINTAINER George Iordanescu <ghiordan@microsoft.com>

RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends \
    gcc g++ \
    wget bzip2 \
    curl apt-transport-https lsb-release gpg \
    git make \
    sudo && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*

ADD aml_sdk_conda_dep_file.yml /tmp/aml_sdk_conda_dep_file.yml
ENV CONDA_DIR /opt/conda
ENV CONDA_ENV_NAME aml-sdk-conda-env    

# reate aml sdk conda env defined by .yml file
RUN /opt/conda/bin/conda env create -q --name $CONDA_ENV_NAME -f /tmp/aml_sdk_conda_dep_file.yml && \
    /opt/conda/bin/conda clean  --yes --all 
ENV CONDA_AUTO_UPDATE_CONDA=false
ENV CONDA_DEFAULT_ENV=$CONDA_ENV_NAME
ENV CONDA_PREFIX=$CONDA_DIR/envs/$CONDA_DEFAULT_ENV
ENV PATH=$CONDA_PREFIX/bin:/opt/conda/bin:$PATH                
        
FROM AzureMLSDKOnly as AzureMLSDKAndAzCli
# Install Azure CLI
ENV AZ_CLI_REPO=stretch   
RUN echo "deb [arch=amd64] https://packages.microsoft.com/repos/azure-cli/ $AZ_CLI_REPO main" | \
    tee /etc/apt/sources.list.d/azure-cli.list && \
    curl -L https://packages.microsoft.com/keys/microsoft.asc | apt-key add - && \
    apt-get update && \
    apt-get install -y --no-install-recommends \
    azure-cli 
    
# Install AzCopy
RUN mkdir -p /tmp/azcopy && mkdir -p /azcopy10 && \
    wget -O /tmp/azcopy/azcopyv10.tar.gz https://aka.ms/downloadazcopy-v10-linux &&  \
    tar -xf /tmp/azcopy/azcopyv10.tar.gz -C /azcopy10 
RUN rm -rf /tmp/azcopy
ENV PATH=/azcopy10/azcopy_linux_amd64_10.1.2:$PATH
    
FROM AzureMLSDKAndAzCli as AzureMLSDKAndAzCliAndAZDevOps
#https://marketplace.visualstudio.com/items?itemName=ms-vsts.cli
RUN az extension add --name azure-devops
        
CMD /bin/bash


Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.55


In [8]:
%dotenv

#add docker image version info only if image_version has been set to a non-empty string, otherwise let docker increment it
image_version_string = os.getenv('image_version')
if image_version_string!="":
    image_version_string = ':'  + image_version_string
    
aml_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
image_version_string
# aml_sudo_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
# 'sudo' + image_version_string
aml_azcli_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
'azcli' + image_version_string
aml_azcli_azdevops_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
'azcliazdevops'  + image_version_string


docker_file_path
aml_docker_image_name
# aml_sudo_docker_image_name
aml_azcli_docker_image_name
aml_azcli_azdevops_docker_image_name

docker_file_dir

working_path = docker_file_dir
! ls -l $working_path


'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.55'

'georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.55'

'georgedockeraccount/aml-sdk_docker_imageazcli:sdk.v1.0.55'

'georgedockeraccount/aml-sdk_docker_imageazcliazdevops:sdk.v1.0.55'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build'

total 8
-rw-rw-r-- 1 loginvm001 loginvm001  321 Aug 16 02:45 aml_sdk_conda_dep_file.yml
-rw-rw-r-- 1 loginvm001 loginvm001 1824 Aug 16 02:45 dockerfile_aml-sdk_docker_image_sdk.v1.0.55


In [9]:
build_command = 'docker build -t ' + aml_docker_image_name  + \
 ' --target AzureMLSDKOnly -f ' + \
docker_file_path +' '+ working_path + ' --no-cache'
build_command

!     $build_command
!     docker build -t $aml_azcli_docker_image_name  --target AzureMLSDKAndAzCli -f $docker_file_path $working_path
!     docker build -t $aml_azcli_azdevops_docker_image_name -f $docker_file_path $working_path


'docker build -t georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.55 --target AzureMLSDKOnly -f /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.55 /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build --no-cache'

Sending build context to Docker daemon  4.608kB
Step 1/11 : FROM continuumio/miniconda3:4.6.14 as AzureMLSDKOnly
 ---> 6b5cf97566c3
Step 2/11 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Running in 9031b48b17d8
Removing intermediate container 9031b48b17d8
 ---> 168b02ca5bf3
Step 3/11 : RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends     gcc g++     wget bzip2     curl apt-transport-https lsb-release gpg     git make     sudo &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
 ---> Running in c5e654919afe
Ign:1 http://deb.debian.org/debian stretch InRelease
Get:2 http://deb.debian.org/debian stretch-updates InRelease [91.0 kB]
Get:3 http://deb.debian.org/debian stretch Release [118 kB]
Get:4 http://deb.debian.org/debian stretch Release.gpg [2434 B]
Get:5 http://security.debian.org/debian-security stretch/updates InRelease [94.3 kB]
Get:6 http://deb.debian.org/debian stretch-updates/main amd64 Packages [27.4 kB]
Get:7 http://deb.debian

debconf: delaying package configuration, since apt-utils is not installed
Fetched 43.6 MB in 0s (89.4 MB/s)
Selecting previously unselected package libpython3.5-minimal:amd64.
(Reading database ... 10409 files and directories currently installed.)
Preparing to unpack .../00-libpython3.5-minimal_3.5.3-1+deb9u1_amd64.deb ...
Unpacking libpython3.5-minimal:amd64 (3.5.3-1+deb9u1) ...
Selecting previously unselected package python3.5-minimal.
Preparing to unpack .../01-python3.5-minimal_3.5.3-1+deb9u1_amd64.deb ...
Unpacking python3.5-minimal (3.5.3-1+deb9u1) ...
Selecting previously unselected package python3-minimal.
Preparing to unpack .../02-python3-minimal_3.5.3-1_amd64.deb ...
Unpacking python3-minimal (3.5.3-1) ...
Selecting previously unselected package mime-support.
Preparing to unpack .../03-mime-support_3.60_all.deb ...
Unpacking mime-support (3.60) ...
Selecting previously unselected package libmpdec2:amd64.
Preparing to unpack .../04-libmpdec2_2.4.2-1_amd64.deb ...
Unpacking li

Selecting previously unselected package sudo.
Preparing to unpack .../37-sudo_1.8.19p1-2.1_amd64.deb ...
Unpacking sudo (1.8.19p1-2.1) ...
Setting up libquadmath0:amd64 (6.3.0-18+deb9u1) ...
Setting up libnpth0:amd64 (1.3-1) ...
Setting up libgomp1:amd64 (6.3.0-18+deb9u1) ...
Setting up libatomic1:amd64 (6.3.0-18+deb9u1) ...
Setting up readline-common (7.0-3) ...
Setting up libcc1-0:amd64 (6.3.0-18+deb9u1) ...
Setting up make (4.1-9.1) ...
Setting up libasan3:amd64 (6.3.0-18+deb9u1) ...
Setting up apt-transport-https (1.4.9) ...
Setting up mime-support (3.60) ...
Setting up sudo (1.8.19p1-2.1) ...
Setting up libcilkrts5:amd64 (6.3.0-18+deb9u1) ...
Setting up libreadline7:amd64 (7.0-3) ...
Setting up libubsan0:amd64 (6.3.0-18+deb9u1) ...
Setting up libtsan0:amd64 (6.3.0-18+deb9u1) ...
Setting up linux-libc-dev:amd64 (4.9.168-1+deb9u5) ...
Setting up distro-info-data (0.36) ...
Setting up libksba8:amd64 (1.3.5-2) ...
Setting up liblsan0:amd64 (6.3.0-18+deb9u1) ...
Setting up libmpx2:amd6

Cache location: /opt/conda/pkgs
Will remove the following tarballs:

/opt/conda/pkgs
---------------
pygments-2.4.2-py_0.tar.bz2                  664 KB
nbformat-4.4.0-py36_0.tar.bz2                141 KB
readline-7.0-h7b6447c_5.tar.bz2              392 KB
xz-5.2.4-h14c3975_4.tar.bz2                  366 KB
numpy-1.16.4-py36h7e9f1db_0.tar.bz2           49 KB
python-dateutil-2.8.0-py36_0.tar.bz2         281 KB
pandoc-2.2.3.2-0.tar.bz2                    20.8 MB
mkl-2019.4-243.tar.bz2                     204.1 MB
ipython-7.7.0-py36h39e3cac_0.tar.bz2         1.1 MB
nbconvert-5.5.0-py_0.tar.bz2                 381 KB
attrs-19.1.0-py36_1.tar.bz2                   56 KB
pickleshare-0.7.5-py36_0.tar.bz2              13 KB
openssl-1.1.1-h7b6447c_0.tar.bz2             5.0 MB
pyrsistent-0.14.11-py36h7b6447c_0.tar.bz2      90 KB
traitlets-4.3.2-py36_0.tar.bz2               133 KB
zlib-1.2.11-h7b6447c_3.tar.bz2               120 KB
cython-0.29.12-py36he6710b0_0.tar.bz2        2.2 MB
certifi-2019.6

 ---> Running in c9a17f0a9109
Removing intermediate container c9a17f0a9109
 ---> 8de85e15644e
Step 11/11 : ENV PATH=$CONDA_PREFIX/bin:/opt/conda/bin:$PATH
 ---> Running in 244c57ca3c37
Removing intermediate container 244c57ca3c37
 ---> d520a928d490
Successfully built d520a928d490
Successfully tagged georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.55
Sending build context to Docker daemon  4.608kB
Step 1/17 : FROM continuumio/miniconda3:4.6.14 as AzureMLSDKOnly
 ---> 6b5cf97566c3
Step 2/17 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Using cache
 ---> 168b02ca5bf3
Step 3/17 : RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends     gcc g++     wget bzip2     curl apt-transport-https lsb-release gpg     git make     sudo &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> e836e261d458
Step 4/17 : ADD aml_sdk_conda_dep_file.yml /tmp/aml_sdk_conda_dep_file.yml
 ---> Using cache
 ---> 90b7443f0a69
Step 5/17 : ENV CONDA

  1200K .......... .......... .......... .......... .......... 15%  123M 0s
  1250K .......... .......... .......... .......... .......... 15% 84.3M 0s
  1300K .......... .......... .......... .......... .......... 16%  128M 0s
  1350K .......... .......... .......... .......... .......... 16% 75.8M 0s
  1400K .......... .......... .......... .......... .......... 17%  122M 0s
  1450K .......... .......... .......... .......... .......... 18% 89.1M 0s
  1500K .......... .......... .......... .......... .......... 18%  135M 0s
  1550K .......... .......... .......... .......... .......... 19% 84.8M 0s
  1600K .......... .......... .......... .......... .......... 19% 88.4M 0s
  1650K .......... .......... .......... .......... .......... 20%  125M 0s
  1700K .......... .......... .......... .......... .......... 21%  109M 0s
  1750K .......... .......... .......... .......... .......... 21%  121M 0s
  1800K .......... .......... .......... .......... .......... 22% 87.1M 0s
  1850K ....

  5400K .......... .......... .......... .......... .......... 65% 77.3M 0s
  5450K .......... .......... .......... .......... .......... 66%  110M 0s
  5500K .......... .......... .......... .......... .......... 67%  122M 0s
  5550K .......... .......... .......... .......... .......... 67%  136M 0s
  5600K .......... .......... .......... .......... .......... 68%  137M 0s
  5650K .......... .......... .......... .......... .......... 68%  102M 0s
  5700K .......... .......... .......... .......... .......... 69%  100M 0s
  5750K .......... .......... .......... .......... .......... 70%  104M 0s
  5800K .......... .......... .......... .......... .......... 70%  109M 0s
  5850K .......... .......... .......... .......... .......... 71% 86.1M 0s
  5900K .......... .......... .......... .......... .......... 71% 73.6M 0s
  5950K .......... .......... .......... .......... .......... 72%  116M 0s
  6000K .......... .......... .......... .......... .......... 73%  138M 0s
  6050K ....

Sending build context to Docker daemon  4.608kB
Step 1/20 : FROM continuumio/miniconda3:4.6.14 as AzureMLSDKOnly
 ---> 6b5cf97566c3
Step 2/20 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Using cache
 ---> 168b02ca5bf3
Step 3/20 : RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends     gcc g++     wget bzip2     curl apt-transport-https lsb-release gpg     git make     sudo &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> e836e261d458
Step 4/20 : ADD aml_sdk_conda_dep_file.yml /tmp/aml_sdk_conda_dep_file.yml
 ---> Using cache
 ---> 90b7443f0a69
Step 5/20 : ENV CONDA_DIR /opt/conda
 ---> Using cache
 ---> 8c6afcb7a7ce
Step 6/20 : ENV CONDA_ENV_NAME aml-sdk-conda-env
 ---> Using cache
 ---> 1db8b125ebd3
Step 7/20 : RUN /opt/conda/bin/conda env create -q --name $CONDA_ENV_NAME -f /tmp/aml_sdk_conda_dep_file.yml &&     /opt/conda/bin/conda clean  --yes --all
 ---> Using cache
 ---> 685c955f464b
Step 8/20 : ENV CONDA_

In [10]:
local_dir = !pwd
crt_local_dir = os.path.join(*([local_dir.s, 'docker_run_dir02']))
crt_local_dir 
!mkdir -p $crt_local_dir
!rm -rf $crt_local_dir/test_*
!ls -l $crt_local_dir
# ! chmod -R ugo=rwx $crt_local_dir

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir02'

total 4
-rw-r--r-- 1 root root 7 Aug 16 02:39 exported_conda_env.yml


In [11]:
# export the resulted conda env
!pwd

cli_command='sudo docker run -it --rm  --name aml-sdk_docker_container ' + \
'-v ' + crt_local_dir + ':/workspace:rw ' + \
aml_docker_image_name + \
' /bin/bash -c "conda env list ; pwd; ls -l /workspace ; '  + \
'conda env export > /workspace/exported_conda_env.yml"'

# docker run -it --rm  --name aml-sdk_docker_container -v /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir:/workspace:rw georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.55 /bin/bash -c "conda env list ; pwd; ls -l /workspace ; conda env export > /workspace/exported_conda_env.yml"

cli_command
! $cli_command

# !docker run  \
#         -v $crt_local_dir:/workspace:rw \
#         $aml_docker_image_name \
#         /bin/bash -c "conda env export > /workspace/exported_conda_env.yml"

/datadrive01/prj/PowerAIWithDocker/amlsdk


'sudo docker run -it --rm  --name aml-sdk_docker_container -v /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir02:/workspace:rw georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.55 /bin/bash -c "conda env list ; pwd; ls -l /workspace ; conda env export > /workspace/exported_conda_env.yml"'

# conda environments:
#
base                     /opt/conda
aml-sdk-conda-env     *  /opt/conda/envs/aml-sdk-conda-env

/
total 4
-rw-r--r-- 1 root root 7 Aug 16 02:39 exported_conda_env.yml

# >>>>>>>>>>>>>>>>>>>>>> ERROR REPORT <<<<<<<<<<<<<<<<<<<<<<

    Traceback (most recent call last):
      File "/opt/conda/lib/python3.7/site-packages/conda/exceptions.py", line 1003, in __call__
        return func(*args, **kwargs)
      File "/opt/conda/lib/python3.7/site-packages/conda_env/cli/main.py", line 73, in do_call
        exit_code = getattr(module, func_name)(args, parser)
      File "/opt/conda/lib/python3.7/site-packages/conda_env/cli/main_export.py", line 94, in execute
        ignore_channels=args.ignore_channels)
      File "/opt/conda/lib/python3.7/site-packages/conda_env/env.py", line 95, in from_environment
        precs = tuple(PrefixGraph(pd.iter_records()).graph)
      File "/opt/conda/lib/python3.7/site-packages/conda/core/prefix_data.py", line 117, in iter_records
      

In [12]:
saved_conda_dependency_file = 'aml_sdk_conda_dep_file'+os.getenv('image_version')+'.yml'
exported_conda_dependency_file = 'aml_sdk_conda_dep_file_exported'+os.getenv('image_version')+'.yml'

!mkdir -p ./docker_history
!cp $docker_file_path ./docker_history/
!cp $conda_dependency_file_path ./docker_history/$saved_conda_dependency_file
!cp {os.path.join(*([crt_local_dir, 'exported_conda_env.yml']))} ./docker_history/$exported_conda_dependency_file



### To run aml SDK, ssh into a vm, run the command below and then go to: 
your_vm.eastus2.cloudapp.azure.com:9001/  
  
Make sure the host port (9001) is open in the VM.  

In [13]:
!echo docker run -it -p 9001:8888 -v $(pwd)/../:/workspace:rw $aml_docker_image_name /bin/bash -c '"jupyter notebook --notebook-dir=/workspace --ip='*' --port=8888 --no-browser --allow-root"'

docker run -it -p 9001:8888 -v /datadrive01/prj/PowerAIWithDocker/amlsdk/../:/workspace:rw georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.55 /bin/bash -c "jupyter notebook --notebook-dir=/workspace --ip=* --port=8888 --no-browser --allow-root"


### Run short tests
Run ' import azureml.core; print("AML SDK version:", azureml.core.VERSION) ' in plain python and in a notebook.
  
Compare the SDK version against latest AML SDK [relese notes](https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes)

In [14]:
!echo docker run  $aml_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'"
!docker run  $aml_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'"

!docker run  $aml_azcli_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'; az --version; azcopy -h"
!docker run  $aml_azcli_azdevops_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'; az --version; az devops -h"


docker run georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.55 /bin/bash -c python -c 'import azureml.core;print(azureml.core.VERSION)'
1.0.55
1.0.55
azure-cli                         2.0.71

command-modules-nspkg               2.0.3
core                              2.0.71
nspkg                              3.0.4
telemetry                          1.0.3

Python location '/opt/az/bin/python3'
Extensions directory '/root/.azure/cliextensions'

Python (Linux) 3.6.5 (default, Aug  8 2019, 03:50:41) 
[GCC 6.3.0 20170516]

Legal docs and information: aka.ms/AzureCliLegal


Your CLI is up-to-date.
/bin/bash: azcopy: command not found
1.0.55
azure-cli                         2.0.71

command-modules-nspkg               2.0.3
core                              2.0.71
nspkg                              3.0.4
telemetry                          1.0.3

Extensions:
azure-devops                      0.12.0

Python location '/opt/az/bin/python3'
Extensions directory '/root/.azure/cliextensions'

Pytho

#### Do the same in a  notebook

In [15]:
test_notebook_base_name = 'test_aml_sdk_docker_image'
test_notebook_name = test_notebook_base_name+'.ipynb'

In [16]:
%%writefile $crt_local_dir/$test_notebook_name

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "AML SDK version: 0.1.68\n"
     ]
    }
   ],
   "source": [
    "# Check core SDK version number\n",
    "import azureml.core\n",
    "\n",
    "print(\"AML SDK version:\", azureml.core.VERSION)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:aml-sdk-conda-env]",
   "language": "python",
   "name": "conda-env-aml-sdk-conda-env-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir02/test_aml_sdk_docker_image.ipynb


In [17]:
!ls -l $crt_local_dir

total 8
-rw-r--r-- 1 root       root         7 Aug 16 02:50 exported_conda_env.yml
-rw-rw-r-- 1 loginvm001 loginvm001 982 Aug 16 02:51 test_aml_sdk_docker_image.ipynb


#### Run the notebook and see the new SDK version


In [18]:
!docker run -v $crt_local_dir:/workspace:rw $aml_docker_image_name /bin/bash -c "jupyter kernelspec list && jupyter nbconvert --ExecutePreprocessor.kernel_name=python3 --execute --to notebook  /workspace/test_aml_sdk_docker_image.ipynb"

Available kernels:
  python3    /opt/conda/envs/aml-sdk-conda-env/share/jupyter/kernels/python3
[NbConvertApp] Converting notebook /workspace/test_aml_sdk_docker_image.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python3
[NbConvertApp] Writing 982 bytes to /workspace/test_aml_sdk_docker_image.nbconvert.ipynb


In [19]:
output_file = test_notebook_base_name+'.nbconvert.ipynb'
!cat $crt_local_dir/$output_file

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "AML SDK version: 1.0.55\n"
     ]
    }
   ],
   "source": [
    "# Check core SDK version number\n",
    "import azureml.core\n",
    "\n",
    "print(\"AML SDK version:\", azureml.core.VERSION)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:aml-sdk-conda-env]",
   "language": "python",
   "name": "conda-env-aml-sdk-conda-env-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.9"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


#### Push image to ([dockerhub](https://hub.docker.com/)) registry (optional step)

In [20]:
# !docker login -u=$docker_login -p=
!docker push $aml_docker_image_name
# !docker push $aml_sudo_docker_image_name
!docker push $aml_azcli_docker_image_name
!docker push $aml_azcli_azdevops_docker_image_name

The push refers to repository [docker.io/georgedockeraccount/aml-sdk_docker_image]

2b4bb40d: Preparing 
102a4989: Preparing 
e5e99e8d: Preparing 
c65c8dc4: Preparing 
2f5d7ee9: Preparing 
b1358a5a: Preparing 
1ff9ade6: Preparing 


2b4bb40d: Pushing  1.881GB/2.684GBAPushing    472MB/2.684GB

2b4bb40d: Pushed   2.737GB/2.684GBsdk.v1.0.55: digest: sha256:e78e96de805af2f613ec289ac963832e88a13e805d00daca65dc29fe2bd9c25e size: 2003
The push refers to repository [docker.io/georgedockeraccount/aml-sdk_docker_imageazcli]

bbe24276: Preparing 
3a19d392: Preparing 
d3124902: Preparing 
2b4bb40d: Preparing 
102a4989: Preparing 
e5e99e8d: Preparing 
c65c8dc4: Preparing 
2f5d7ee9: Preparing 
b1358a5a: Preparing 
1ff9ade6: Preparing 
d3124902: Pushed   459.2MB/429MBBeraccount/aml-sdk_docker_image [Ksdk.v1.0.55: digest: sha256:a8223f3d8b66dd47978c7ef8ea077776f86c30d175072102ebcdb86ad18dda57 size: 2635
The push refers to repository [docker.io/georgedockeraccount/aml-sdk_docker_imageazcliazdevops]

f210f741: Preparing 
bbe24276: Preparing 
3a19d392: Preparing 
d3124902: Preparing 
2b4bb40d: Preparing 
102a4989: Preparing 
e5e99e8d: Preparing 
c65c8dc4: Preparing 
2f5d7ee9: Preparing 
b1358a5a: Preparing 
1ff9ade6: Preparing 
210f741: Pushed   17.08MB/15.69MBraccount/aml-sdk_docker_imageazc

In [21]:
!jupyter nbconvert --to html createAMLSDKDocker.ipynb

[NbConvertApp] Converting notebook createAMLSDKDocker.ipynb to html
[NbConvertApp] Writing 519485 bytes to createAMLSDKDocker.html
